In [1]:
# main.py
# Cross-asset relative value trading loop (LIVE TEST VERSION)

import time
import pandas as pd

from data_layer_v2 import IBKRDataClient
from feature_layer_v2 import add_intraday_features, add_cross_asset_spread_features
from exec_layer_v2 import (
    StrategyParams,
    ExecParams,
    init_state,
    decide_portfolio_target,
    Executor
)

# ==========================================================
# 1️⃣ 初始化
# ==========================================================

SYMBOL_A = "SPY"
SYMBOL_B = "TLT"
symbols = [SYMBOL_A, SYMBOL_B]

client = IBKRDataClient()
contracts = client.qualify(symbols)

strategy_params = StrategyParams()   # 使用我们修改后的低门槛参数
exec_params = ExecParams()

state = init_state()
executor = Executor(client.ib, exec_params)

print("System started...")

# ==========================================================
# 2️⃣ 主循环
# ==========================================================

start = time.time()
RUN_SECONDS = 300  # 运行 5 分钟测试

while time.time() - start < RUN_SECONDS:

    # 1️⃣ 获取分钟数据
    data_dict = client.get_minute_bars_multi(contracts)

    df_a = data_dict[SYMBOL_A]
    df_b = data_dict[SYMBOL_B]

    # 2️⃣ 单资产特征
    df_a = add_intraday_features(df_a)
    df_b = add_intraday_features(df_b)

    # 3️⃣ 跨资产特征
    spread_df = add_cross_asset_spread_features(df_a, df_b)

    if spread_df.empty:
        print("Spread DF empty, skipping...")
        time.sleep(60)
        continue

    last_row = spread_df.iloc[-1]

    print("Current z_spread:", last_row["z_spread"])
    print("Entry threshold:", strategy_params.z_entry)
    print("In trade:", state["in_trade"])

    # ======================================================
    # 4️⃣ 组合级决策（⚠️ 正确调用方式）
    # ======================================================

    targets = decide_portfolio_target(
        spread_df,            # 传入完整 features DF
        state,
        strategy_params,
        exec_params,
        pd.Timestamp.now()
    )

    print("Decision:", targets)

    # ======================================================
    # 5️⃣ 分腿执行
    # ======================================================

    for leg in targets:

        if leg == "A":
            symbol = SYMBOL_A
        elif leg == "B":
            symbol = SYMBOL_B
        else:
            continue

        contract = contracts[symbol]
        target_position = targets[leg]

        executor.trade_to_target(
            symbol,
            contract,
            target_position,
            "LiveSignal"
        )

    time.sleep(60)  # 1分钟更新一次

# ==========================================================
# 3️⃣ 结束
# ==========================================================

client.disconnect()
print("System stopped.")

System started...


Error 10349, reqId 38: Order TIF was set to DAY based on order preset.
Canceled order: Trade(contract=Stock(conId=756733, symbol='SPY', exchange='SMART', primaryExchange='ARCA', currency='USD', localSymbol='SPY', tradingClass='SPY'), order=MarketOrder(orderId=38, clientId=1, action='BUY', totalQuantity=50), orderStatus=OrderStatus(orderId=38, status='Cancelled', filled=0.0, remaining=0.0, avgFillPrice=0.0, permId=0, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2026, 2, 13, 20, 45, 3, 537566, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0), TradeLogEntry(time=datetime.datetime(2026, 2, 13, 20, 45, 3, 575533, tzinfo=datetime.timezone.utc), status='Cancelled', message='Error 10349, reqId 38: Order TIF was set to DAY based on order preset.', errorCode=10349)], advancedError='')


Current z_spread: -3.1099292181573372
Entry threshold: 0.3
In trade: False
Current z: -3.1099292181573372
Entry threshold: 0.3
In trade: False
>>> ENTER LONG A / SHORT B
Decision: {'A': 50, 'B': -50}
Sending order: BUY 50 SPY | Reason: LiveSignal


Error 10349, reqId 39: Order TIF was set to DAY based on order preset.
Canceled order: Trade(contract=Stock(conId=15547841, symbol='TLT', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='TLT', tradingClass='NMS'), order=MarketOrder(orderId=39, clientId=1, action='SELL', totalQuantity=50), orderStatus=OrderStatus(orderId=39, status='Cancelled', filled=0.0, remaining=0.0, avgFillPrice=0.0, permId=0, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2026, 2, 13, 20, 45, 4, 539587, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0), TradeLogEntry(time=datetime.datetime(2026, 2, 13, 20, 45, 4, 544929, tzinfo=datetime.timezone.utc), status='Cancelled', message='Error 10349, reqId 39: Order TIF was set to DAY based on order preset.', errorCode=10349)], advancedError='')


Order submitted for SPY.
Sending order: SELL 50 TLT | Reason: LiveSignal
Order submitted for TLT.
Current z_spread: -2.5908689984924598
Entry threshold: 0.3
In trade: True
Current z: -2.5908689984924598
Entry threshold: 0.3
In trade: True
Decision: {}
Current z_spread: -1.962934048057113
Entry threshold: 0.3
In trade: True
Current z: -1.962934048057113
Entry threshold: 0.3
In trade: True
Decision: {}
Current z_spread: -1.4386364922637618
Entry threshold: 0.3
In trade: True
Current z: -1.4386364922637618
Entry threshold: 0.3
In trade: True
Decision: {}
Current z_spread: -1.2755436628713268
Entry threshold: 0.3
In trade: True
Current z: -1.2755436628713268
Entry threshold: 0.3
In trade: True
Decision: {}
System stopped.
